<a href="https://colab.research.google.com/github/joony0512/Deep_Learning_Class/blob/main/Part6/P6_Ch01_CH03_06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TensorBoard Visualization


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import math

import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.losses as losses


In [3]:

def iou(y_true, y_pred):
    smooth = 1.
    # Flatten
    y_true = tf.reshape(y_true, [-1])
    y_pred = tf.reshape(y_pred, [-1])

    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection

    score = intersection / (union + smooth)
    return score

def dice_coef(y_true, y_pred):
    smooth = 1.
    # Flatten
    y_true = tf.reshape(y_true, [-1])
    y_pred = tf.reshape(y_pred, [-1])

    intersection = tf.reduce_sum(y_true * y_pred)
    score = (2. * intersection )/ ( tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

    return score

def dice_loss(y_true, y_pred):
    loss = 1 - dice_coef(y_true, y_pred)

    return loss

def bce_dice_loss(y_true, y_pred):
    loss = 1. * losses.binary_crossentropy(y_true, y_pred) + 1. * dice_loss(y_true, y_pred)

    return loss


In [4]:
import albumentations as A
import cv2

class Augmentation:
  def __init__(self, size, mode ='train'):
    if mode =='train':
      # Declare an augmentation pipeline
      self.transform = A.Compose([
          A.HorizontalFlip(p=0.5), # 좌우반전, p(적용할 확률)
          A.ShiftScaleRotate(
              p = 0.5,
              shift_limit = 0.05,
              scale_limit = 0.05,
              rotate_limit = 15,
          ), # 이동%, 확대축소%, 회전각도 제약걸기
          A.CoarseDropout(
              p = 0.5,
              max_holes =8,
              max_height = int(0.1 * size),
              max_width = int(0.1 * size),
          ),
          A.RandomBrightnessContrast(p=0.2),
      ])

  def __call__(self, **kwargs):
    if self.transform:
      aumented = self.transform(**kwargs)
      img = aumented['image']
      mask = aumented['mask']

      return img, mask


In [5]:
class DataGenerator(keras.utils.Sequence):
    def __init__(
        self,
        batch_size,
        csv_path,
        fold,
        image_size,
        mode ='train',
        shuffle = True):

        self.batch_size = batch_size
        self.image_size = image_size
        self.fold = fold
        self.mode = mode
        self.shuffle = shuffle

        self.df = pd.read_csv(csv_path)

        if self.mode == 'train':
            self.df = self.df[self.df['fold'] != self.fold]
        elif self.mode =='val':
            self.df = self.df[self.df['fold'] == self.fold]


        ### Remove invalid files
        ### https://github.com/tensorflow/models/issues/3134
        invalid_filenames = [
            'Egyptian_Mau_14',
            'Egyptian_Mau_139',
            'Egyptian_Mau_145',
            'Egyptian_Mau_156',
            'Egyptian_Mau_167',
            'Egyptian_Mau_177',
            'Egyptian_Mau_186',
            'Egyptian_Mau_191',
            'Abyssinian_5',
            'Abyssinian_34',
            'chihuahua_121',
            'beagle_116'
        ]
        self.df = self.df[~self.df['filename'].isin(invalid_filenames)]
        self.transform = Augmentation(image_size, mode)

        self.on_epoch_end()


    def __len__(self):
        return math.ceil(len(self.df)/self.batch_size)

    def __getitem__(self, idx):
        strt = idx * self.batch_size
        fin = (idx + 1)* self.batch_size
        data = self.df.iloc[strt:fin]

        batch_x, batch_y = self.get_data(data)

        return np.array(batch_x), np.array(batch_y)

    def get_data(self, data):
        batch_x =[]
        batch_y =[]

        for _, r in data.iterrows():
            file_name = r['filename']

            image = cv2.imread(f'/content/drive/MyDrive/data/images/{file_name}.jpg')
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # input image resize (배치로 묶기위해)
            image = cv2.resize(image, (self.image_size, self.image_size))

            # mask가 Label 로 사용
            mask = cv2.imread(f'/content/drive/MyDrive/data/annotations/trimaps/{file_name}.png',
                              cv2.IMREAD_GRAYSCALE)
            mask = cv2.resize(mask, (self.image_size, self.image_size))
            mask[mask != 1] =0

            if self.mode == 'train':
            #   image = image.astype('uint8')
              image, mask= self.transform(image = image,
                                     mask = mask)

            # rescaling
            image = image.astype('float32')
            image = image /255.

            mask = mask.astype('float32')

            # mask가 Label 로 사용
            batch_x.append(image)
            batch_y.append(mask)
        return batch_x, batch_y



    def on_epoch_end(self): # callback function
        if self.shuffle:
            self.df = self.df.sample(frac =1).reset_index(drop=True)

csv_path = '/content/drive/MyDrive/data/kfolds.csv'
train_generator = DataGenerator(
    batch_size = 64,
    csv_path =csv_path,
    fold = 1,
    image_size = 128,
    mode ='train',
    shuffle = True
)

val_generator = DataGenerator(
    batch_size = 64,
    csv_path =csv_path,
    fold = 1,
    image_size = 128,
    mode ='val',
    shuffle = True
)

In [6]:
!pip install keras_unet

In [7]:
#https://github.com/karolzak/keras-unet
from keras_unet.models import custom_unet

model = custom_unet(
    input_shape=(128, 128, 3),
    use_batch_norm=True,
    upsample_mode= 'deconv',
    dropout_type='spatial',
    use_attention = True,
    num_classes=1,
    filters=64,
    num_layers =4,
    dropout=0.2,
    output_activation='sigmoid')

model.compile(optimizer = 'adam', loss = bce_dice_loss, metrics =[iou])
model.summary()

-----------------------------------------
keras-unet init: TF version is >= 2.0.0 - using `tf.keras` instead of `Keras`
-----------------------------------------
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128, 128, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 128, 128, 64)         1728      ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 128, 128, 64)         256       ['conv2d[0][0]']              
 Normalization)                                                                                   
                               

In [8]:
def plot_to_image(figure):
  """Converts the matplotlib plot specified by 'figure' to a PNG image and
  returns it. The supplied figure is closed and inaccessible after this call."""
  # Save the plot to a PNG in memory.
  buf = io.BytesIO()
  plt.savefig(buf, format='png')
  # Closing the figure prevents it from being displayed directly inside
  # the notebook.
  plt.close(figure)
  buf.seek(0)
  # Convert PNG buffer to TF image
  image = tf.image.decode_png(buf.getvalue(), channels=4)
  # Add the batch dimension
  image = tf.expand_dims(image, 0)
  return image

def image_grid(img, mask, preds):
  figure = plt.figure(figsize=(10,10))
  for i in range(3):
    # Start next subplot.
    plt.subplot(3, 3, 3*i + 1)
    plt.imshow(img[i])
    plt.axis('off')
    plt.title('img')

    plt.subplot(3,3,3*i + 2)
    plt.imshow(mask[i], cmap = 'gray')
    plt.axis('off')
    plt.title('gt')

    plt.subplot(3,3,3*i + 3)
    plt.imshow(preds[i, ... , 0], cmap = 'gray')
    plt.axis('off')
    plt.title('preds')

  return figure


In [9]:
from tensorflow.keras.callbacks import Callback

class TrainHistory(Callback):
  def __init__(self, data = None, log_dir = 'weights'):
    self.img, self.mask = data
    self.writer = tf.summary.create_file_writer(log_dir)

  def on_epoch_end(self, epoch, logs = None):
    pred = self.model.predict(self.img)

    figure = image_grid(self.img, self.mask, pred)
    with self.writer.as_default():
      tf.summary.image('plot', plot_to_image(figure), epoch)

In [10]:
from tensorflow.keras.callbacks import TensorBoard

callbacks = [
    TensorBoard('weights/'),
    TrainHistory(log_dir = 'weights/', data = val_generator[0][:3])
]

In [ ]:
history = model.fit(
    train_generator ,
    validation_data = val_generator,
    epochs = 10,
    verbose =1,
    callbacks = callbacks
)

In [ ]:
%load_ext tensorboard
%tensorboard --log_dir weights/